In [1]:
!Nvidia-smi

/usr/bin/zsh: /home/dimask/miniconda3/lib/libtinfo.so.6: no version information available (required by /usr/bin/zsh)
Wed Mar 15 15:18:57 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.05    Driver Version: 528.24       CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:04:00.0  On |                  N/A |
|  0%   49C    P8    40W / 420W |   1038MiB / 24576MiB |     13%      Default |
|                               |                      |                  N/A |
+-------------------------------+------------------

<h2>1. Model And Installation Tensorflow</h2>
we can download of pretrained Model from tensorflow collection model on tfhub. example of pretrained model that we can explore visit <a href="https://tfhub.dev/tensorflow/collections/object_detection/1" title="Pretrained Model">tfhub.dev</a> for this tutorial we used pretained model called ssd-mobilenet-v2 </p> </blockquote>

In [2]:
import os

In [3]:
PRETRAINED_MODEL_NAME = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'
PRETRAINED_MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz'

In [4]:
paths = {'APIMODEL_PATH': os.path.join('Tensorflow','models'),
         'PRETRAINED_MODEL_PATH': os.path.join('Tensorflow', 'workspace','pre-trained-models'),}
!mkdir -p {paths['APIMODEL_PATH']}
!mkdir -p {paths['PRETRAINED_MODEL_PATH']}

if not os.path.exists(os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection')):
    !git clone https://github.com/tensorflow/models {paths['APIMODEL_PATH']}

if not os.path.exists(paths['PRETRAINED_MODEL_PATH']):
    !wget {PRETRAINED_MODEL_URL}
    !mv {PRETRAINED_MODEL_NAME+'.tar.gz'} {paths['PRETRAINED_MODEL_PATH']}
    !cd {paths['PRETRAINED_MODEL_PATH']} && tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}

/usr/bin/zsh: /home/dimask/miniconda3/lib/libtinfo.so.6: no version information available (required by /usr/bin/zsh)
/usr/bin/zsh: /home/dimask/miniconda3/lib/libtinfo.so.6: no version information available (required by /usr/bin/zsh)


In [5]:
!cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && cp object_detection/packages/tf2/setup.py . && python -m pip install . 

/usr/bin/zsh: /home/dimask/miniconda3/lib/libtinfo.so.6: no version information available (required by /usr/bin/zsh)
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Processing /mnt/c/Users/Dimas K/Documents/GitHub/TF-GPU-Windows/Tensorflow/models/research
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 3.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 352.1/352.1 kB 6.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 3.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 3.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 5.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.9/26.9 MB 3.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 5.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━

<h3>2. Vertify GPU Detected for Tensorflow</h3>

In [7]:
!{python3 -c "import tensorflow as tf; print(tf.config.list_physical_devices('GPU'))"}

/usr/bin/zsh: /home/dimask/miniconda3/lib/libtinfo.so.6: no version information available (required by /usr/bin/zsh)
2023-03-15 20:13:16.896651: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/dimask/miniconda3/lib/:/home/dimask/miniconda3/envs/tfod3/lib/
2023-03-15 20:13:16.901494: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/dimask/miniconda3/lib/:/home/dimask/miniconda3/envs/tfod3/lib/
2023-03-15 20:13:16.901683: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the miss

<h4>2.d Vertify Installation of Tensorflow Object Detection API</h4>

In [8]:
VERIFICATION_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'builders', 'model_builder_tf2_test.py')
# Verify Installation
!python {VERIFICATION_SCRIPT}

/usr/bin/zsh: /home/dimask/miniconda3/lib/libtinfo.so.6: no version information available (required by /usr/bin/zsh)
2023-03-16 07:55:10.207052: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/dimask/miniconda3/lib/:/home/dimask/miniconda3/envs/tfod3/lib/
2023-03-16 07:55:10.209721: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/dimask/miniconda3/lib/:/home/dimask/miniconda3/envs/tfod3/lib/
2023-03-16 07:55:10.209799: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the miss